In [22]:
import psycopg2
import json
import numpy
import pandas as pd

In [4]:
with open("../../elections_login.json", "rb") as f:
    login_info = json.load(f)

In [10]:
conn = psycopg2.connect(host=login_info["ip"]
                       ,dbname=login_info["db"]
                       ,user=login_info["username"]
                       ,password=login_info["password"])

In [11]:
cursor = conn.cursor()

In [20]:
cursor.execute("""SELECT table_name FROM information_schema.tables
       WHERE table_schema = 'public'""")
tables = [x[0] for x in cursor.fetchall()]

In [23]:
table_dfs = {}
for table in tables:
    table_dfs[table] = pd.read_sql("SELECT * FROM {0}".format(table), conn)

In [26]:
table_dfs.keys()

dict_keys(['ballots', 'committee_history', 'transaction_details', 'committees_list', 'donor', 'election_activity', 'payee', 'statement_of_org', 'transactions'])

In [38]:
table_dfs['donor']['donor_cats'][0]['categories'][0]

'trash'